Esse notebook tem como finalidade explorar o funcionamento do Website da Prefeitura de Caçador para a extração dos dados de noticias de saúde.

Next Steps:

1. Trabalhar com a paginação das noticias anteriores ao header da página principal.
2. Criação da classe de ingestão de dados.

In [ ]:
import os
import json
import boto3
import requests
from time import sleep
from bs4 import BeautifulSoup
from pytz import timezone
from datetime import datetime

In [ ]:
AWS_BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")
AWS_LANDING_PATH = os.getenv("AWS_LANDING_PATH")

### Coletar os Posts nível Header

In [3]:
website_painel_noticias = "https://cacador.sc.gov.br/pagina-27075-2/pagina-8577/pagina-8577-not/"

response = requests.get(website_painel_noticias)
soup = BeautifulSoup(response.text, "html.parser")
posts = soup.find("div", {"class": "posts"})
posts = posts.find_all("div", {"class": "post-item"})

In [4]:
posts_header = []

for post in posts:
    post_link = post.find("a", {"class": "link"}).attrs["href"]
    post_title = " ".join(post.find("h6").text.split())

    posts_header.append({"link": post_link, "title": post_title})


In [5]:
posts_header[:3]

[{'link': 'https://cacador.sc.gov.br/saude-de-cacador-implementa-o-planificasus-para-a-qualificacao-do-atendimento-ao-publico/',
  'title': 'Saúde de Caçador implementa o PlanificaSUS, para a qualificação do atendimento ao público'},
 {'link': 'https://cacador.sc.gov.br/prefeitura-de-cacador-realiza-o-dia-d-da-campanha-de-vacinacao-contra-a-gripe-no-dia-10-de-maio/',
  'title': 'Prefeitura de Caçador realiza o Dia D da Campanha de Vacinação contra a gripe, no dia 10 de maio'},
 {'link': 'https://cacador.sc.gov.br/prefeitura-de-cacador-implanta-atualizacao-no-protocolo-de-acolhimento-de-risco-na-upa/',
  'title': 'Prefeitura de Caçador implanta atualização no protocolo de acolhimento de risco na UPA'}]

### Coletar os Detalhes dos Posts

In [6]:
posts_details = []
for post in posts_header:
    response = requests.get(post["link"])
    soup = BeautifulSoup(response.text, "html.parser")

    post_date = soup.find("span", {"class": "posted-on"}).time.attrs["datetime"]
    post_content = " ".join(soup.find("div", {"class": "entry-content clearfix"}).text.split())

    posts_details.append(
        post | {
            "posted_on": post_date,
            "content": post_content
        }
    )

    sleep(2)

posts_details = [p | {"scrap_date": datetime.now(timezone("America/Sao_Paulo")).strftime("%Y-%m-%dT%H:%M:%S")} for p in posts_details]

In [7]:
posts_details[:3]

[{'link': 'https://cacador.sc.gov.br/saude-de-cacador-implementa-o-planificasus-para-a-qualificacao-do-atendimento-ao-publico/',
  'title': 'Saúde de Caçador implementa o PlanificaSUS, para a qualificação do atendimento ao público',
  'posted_on': '2025-04-29T18:15:01-03:00',
  'content': 'A Prefeitura de Caçador, através da Secretaria de Saúde, iniciou o PlanificaSUS, que é um projeto do Ministério da Saúde para a qualificação dos atendimentos nas unidades de Saúde do município. Com apoio do Hospital Israelita Albert Einstein, o programa está ajudando as Equipes de Saúde da Família (ESFs) a organizar o trabalho. Nesta primeira etapa, chamada de Ciclo 1, as equipes estão participando de treinamentos para: Organizar melhor as famílias atendidas por cada equipe; Garantir mais segurança no atendimento; Fortalecer o relacionamento com a comunidade. O objetivo é facilitar o acesso, resolver problemas com mais eficiência e oferecer cuidados mais próximos das necessidades de cada pessoa.',
  

#### Enviar os dados ao S3

In [ ]:
client = boto3.client(
    "s3",
    region_name="us-east-2",
)

file_name = "test_" + datetime.now(timezone("America/Sao_Paulo")).strftime("%Y%m%d_%H%M%S")

client.put_object(
    Bucket=AWS_BUCKET_NAME,
    Key=f"{AWS_LANDING_PATH}/2025/01/02/{file_name}.json",
    Body=json.dumps(posts_details)
)

#### Classe de Ingestão de Dados

In [99]:
class ScrapyData():
    def __init__(
        self
    ):
        pass

    def scrapy_post_header(self):
        ...

    def scrapy_post_detail(self):
        ...

    def execute_pipeline(self):
        ...

class S3Ingestion(ScrapyData):
    ...